In [1]:
# 개빡치네

In [1]:
from elasticsearch import Elasticsearch
from datasets import load_from_disk, DatasetDict, Dataset
from tqdm import tqdm

In [2]:
es = Elasticsearch('http://localhost:30001')
es.info()

{'name': '_eN3Dqz',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'vExZ3cZVR7Gms-wk7OVygA',
 'version': {'number': '5.4.3',
  'build_hash': 'eed30a8',
  'build_date': '2017-06-22T00:34:03.743Z',
  'build_snapshot': False,
  'lucene_version': '6.5.1'},
 'tagline': 'You Know, for Search'}

In [3]:
INDEX = 'wiki'

In [5]:
test = load_from_disk("/opt/ml/input/data/test_dataset")
test['validation']

Dataset({
    features: ['id', 'question'],
    num_rows: 600
})

In [11]:
contexts = []
for i in range(len(test['validation'])):
    question = test['validation'][i]['question']
    context = ''
    body = {
        'query': {
            'match': {
                'text': question
            }
        }
    }
    result = es.search(index=INDEX, body=body)
    for j in range(len(result['hits']['hits'])):
        candidate = result['hits']['hits'][j]['_source']['text']
        context += candidate + '@@@'
        
    contexts.append(context)
    
new_test = {'question': test['validation']['question'],
            'id': test['validation']['id'],
            'context': contexts
           }

In [ ]:
new_data = Dataset.from_dict(new_test)
new_datasets = DatasetDict({"validation":new_test})
new_datasets.save_to_disk("/opt/ml/input/data/es_retrieval")